In [1]:
import pandas as pd
import torch

In [2]:
from transformers import AutoTokenizer, AutoModelWithLMHead

tokenizer = AutoTokenizer.from_pretrained("mrm8488/t5-base-finetuned-emotion")

model = AutoModelWithLMHead.from_pretrained("mrm8488/t5-base-finetuned-emotion")

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thouroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565
C:\Users\User\Desktop\programowanie_web_etc\python_projects\data_projects\venv\lib\site-packages\transformers\models\auto\modeling_auto.py:1564: FutureWarning: The class `AutoModelWithLMHead` is deprecated and will be removed in a future version. Please use `AutoModelForCausalLM` for causal language models, `AutoModelForMaskedLM` for masked language models and `AutoModelForSeq2SeqLM` for encoder-decoder models.
  warnings.warn(


pytorch_model.bin:   0%|          | 0.00/892M [00:00<?, ?B/s]

C:\Users\User\Desktop\programowanie_web_etc\python_projects\data_projects\venv\lib\site-packages\huggingface_hub\file_download.py:149: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\User\.cache\huggingface\hub\models--mrm8488--t5-base-finetuned-emotion. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


In [3]:
def get_emotion(text):
  input_ids = tokenizer.encode(text + '</s>', return_tensors='pt')

  output = model.generate(input_ids=input_ids,
               max_length=2)
  
  dec = [tokenizer.decode(ids) for ids in output]
  label = dec[0]
  return label

In [5]:
get_emotion("i feel as if i havent blogged in ages are at least truly blogged i am doing an update cute") # Output: 'joy'

'<pad> joy'

In [6]:
get_emotion("i have a feeling i kinda lost my best friend") # Output: 'sadness'

'<pad> sadness'

In [7]:
df_path = 'dataframes/with_counts/combined_count.csv'

In [8]:
df = pd.read_csv(df_path, index_col = 0)

In [9]:
sample_lyrics = df.sample(5)['Song Lyrics'].str.lower()

In [13]:
import re
emotion_regex = re.compile('<pad>\s(.*)')
emotion_regex.findall('<pad> sadness')[0]

'sadness'

In [10]:
for song in sample_lyrics:
    song_emotion = get_emotion(song)
    print('song:')
    print(song)
    print('===================================')
    print('emotion:')
    print(song_emotion)
    print('===================================')
    print('===================================')

song:
 creeping round my door i got a big surprise couldnt see a thing but open skies theyve taken her away where is she now  wish that she was here i wish shed hold my hand maybe she would laugh maybe understand why was i so cruel where is she now   acting like a fool i had to make her cry maybe im a leo but i aint a lion im hurtin oh so bad i want her nowyou might also like
emotion:
<pad> anger
song:
 get up get up get up get up wake up wake up wake up wake up oh baby now lets get down tonight ooh  baby im hot just like an oven heal me my darling i need some lovin heal me my darling and baby i cant hold it much longer heal me my darling its getting stronger and stronger heal me my darling  and when i get that feeling i want sexual healing sexual healing oh baby makes me feel so fine helps to relieve my mind sexual healing baby is good for me sexual healing is something thats good for me  whenever blue teardrops are falling and my emotional stability is leaving me there is something i

Token indices sequence length is longer than the specified maximum sequence length for this model (637 > 512). Running this sequence through the model will result in indexing errors


song:
  you wanna know if i know why i cant say that i do dont understand the evil eye or how one becomes two and i just cant recall what started it all or how to begin in the end i aint here to break it just see how far it will bend again and again again and again  i wanna make it i wanna make it wit chu anytime anywhere i wanna make it again and again i wanna make it wit chu  sometimes the same is different but mostly its the same these mysteries of life that jus aint my thing if i told you that i knew about the sun and the moon id be untrue the only thing i know for sure is what i wanna do anytime anywhere and i say babe  i wanna make it again and again i wanna make it wit chu anytime anywhere i wanna make it again and again i wanna make it wit chu i wanna make it anytime anywhere i wanna make it wit chu yeah i wanna make it oohooh i wanna make it wit chu you might also like  i wanna make it again and again i wanna make it wit chu i wanna make it anytime anywhere i wanna make it wit

In [14]:
def assign_emotion(row):
    lyrics = row['Song Lyrics']
    emotion = get_emotion(lyrics)
    emotion = emotion_regex.findall(emotion)[0]
    return emotion

In [17]:
def apply_emotion(df):
    df['emotion'] = df.apply(
        lambda row: assign_emotion(row),
        axis = 1
    )
    return df

In [18]:
df = apply_emotion(df)

In [19]:
df.sample(5)

,Artist,Song Title,Full Title,Release Date,Year,Month,Day,Pageviews,url,featured_count,...,manual_love_word_percent,manual_money_word_percent,manual_violence_word_percent,manual_drugs_word_percent,manual_gendered_word_percent,manual_sadness_word_percent,manual_joy_word_percent,manual_yes_word_percent,manual_no_word_percent,emotion
287,JAY-Z,Renegade,Renegade by JAY-Z (Ft. Eminem),2001-09-11,2001.0,9.0,11.0,1362662,https://genius.com/Jay-z-renegade-lyrics,1,...,0.000,0.258,0.000,0.000,14.470,0.775,0.517,0.000,2.842,joy
392,Yasiin Bey,Sunshine,Sunshine by Yasiin Bey,2004-10-12,2004.0,10.0,12.0,50368,https://genius.com/Yasiin-bey-sunshine-lyrics,0,...,1.034,0.258,1.809,0.000,23.514,0.258,0.000,0.775,3.618,joy
475,Fugazi,Epic Problem,Epic Problem by Fugazi,2001-10-16,2001.0,10.0,16.0,11078,https://genius.com/Fugazi-epic-problem-lyrics,0,...,0.000,0.000,3.502,0.000,2.724,0.000,0.000,0.000,3.502,sadness
335,Kendrick Lamar,LOYALTY.,LOYALTY. by Kendrick Lamar (Ft. Rihanna),2017-04-14,2017.0,4.0,14.0,2934448,https://genius.com/Kendrick-lamar-loyalty-lyrics,1,...,1.292,0.775,0.000,0.258,9.561,0.000,0.258,0.000,3.618,love
28,Beyoncé,Pray You Catch Me,Pray You Catch Me by Beyoncé,2016-04-23,2016.0,4.0,23.0,1314584,https://genius.com/Beyonce-pray-you-catch-me-l...,0,...,0.219,0.000,0.000,0.000,1.532,0.000,0.219,0.000,0.438,sadness


In [21]:
df.to_csv('dataframes/with_counts/combined_count.csv')